In [ ]:
import os
import symlib
import pandas as pd
from astropy.constants import G
from astropy import units as u
import numpy as np

base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
suite_names = ["SymphonyLMC", "SymphonyMilkyWay", "SymphonyGroup", "SymphonyLCluster", "SymphonyCluster"]


In [ ]:
def save_virial_radius(input_dir,suite_name):
    output_dir = os.path.join(base_dir, "output", suite_name)

    n_halos = symlib.n_hosts(suite_name)
    halo_ids, rvir_list = [], []

    for halo_idx in range(n_halos):
        sim_dir = symlib.get_host_directory(input_dir, suite_name, halo_idx)
        try:
            r, _ = symlib.read_rockstar(sim_dir)
            rvir = r[0, -1]["rvir"] # [kpc]
            halo_ids.append(f'{halo_idx:03d}')
            rvir_list.append(rvir)
        except FileNotFoundError:
            print(f"[Warning] Rockstar file not found for Halo {halo_idx}")
            continue

    df = pd.DataFrame({"halo_id": halo_ids, "rvir": rvir_list})
    df.to_csv(os.path.join(output_dir, "virial_radius.csv"), index=False)
    print(f"[Saved] Radius CSV to {output_dir}/virial_radius.csv")

In [ ]:
def save_virial_velocity(input_dir, suite_name):
    output_dir = os.path.join(base_dir, "output", suite_name)

    n_halos = symlib.n_hosts(suite_name)
    halo_ids, vvir_list = [], []

    for halo_idx in range(n_halos):
        sim_dir = symlib.get_host_directory(input_dir, suite_name, halo_idx)
        try:
            r, _ = symlib.read_rockstar(sim_dir)
            host = r[0, -1]
            v_host = host['v'][-1]         # Halo bulk velocity [km/s]
            r_vir = host['rvir']           # Virial radius [kpc]
            m_vir = host['m']              # Virial mass [Msun]
            m = m_vir * u.Msun
            r = r_vir * u.kpc
            G_kpc = G.to(u.kpc * (u.km/u.s)**2 / u.Msun)
            vvir = np.sqrt(G_kpc * m / r).to(u.km / u.s).value  # Virial velocity [km/s]

            halo_ids.append(f'{halo_idx:03d}')
            vvir_list.append(vvir)
        except FileNotFoundError:
            print(f"[Warning] Rockstar file not found for Halo {halo_idx}")
            continue

    df = pd.DataFrame({"halo_id": halo_ids, "vvir": vvir_list})
    df.to_csv(os.path.join(output_dir, "virial_velocity.csv"), index=False)
    print(f"[Saved] Velocity CSV to {output_dir}/virial_velocity.csv")


In [ ]:
def save_rvmax(input_dir,suite_name):
    output_dir = os.path.join(base_dir, "output", suite_name)

    n_halos = symlib.n_hosts(suite_name)
    halo_ids, rvmax_list = [], []

    for halo_idx in range(n_halos):
        sim_dir = symlib.get_host_directory(input_dir, suite_name, halo_idx)
        try:
            r, _ = symlib.read_rockstar(sim_dir)
            rvmax = r[0, -1]["rvmax"] # [kpc]
            halo_ids.append(f'{halo_idx:03d}')
            rvmax_list.append(rvmax)
        except FileNotFoundError:
            print(f"[Warning] Rockstar file not found for Halo {halo_idx}")
            continue

    df = pd.DataFrame({"halo_id": halo_ids, "rvmax": rvmax_list})
    df.to_csv(os.path.join(output_dir, "max_radius.csv"), index=False)
    print(f"[Saved] Radius CSV to {output_dir}/max_radius.csv")


In [ ]:
def compute_rvmax_from_mass_profile(r, m):
    """Compute the radius where the circular velocity is maximal."""
    vc = np.sqrt(m / r)
    idx_max = np.argmax(vc)
    return r[idx_max]

def save_rvmax_from_mass_profile(base_dir, suite_name):
    import os
    import numpy as np
    import pandas as pd

    mass_dir = os.path.join(base_dir, "output", suite_name, "mass_profiles")
    df_rvir = pd.read_csv(os.path.join(base_dir, "output", suite_name, "virial_radius.csv"),
                          dtype={"halo_id": str}) 
    output_dir = os.path.join(base_dir, "output", suite_name)
    os.makedirs(output_dir, exist_ok=True)

    files = sorted([f for f in os.listdir(mass_dir) if f.endswith(".csv")])
    halo_ids, rvmax_list = [], []

    for f in files:
        try:
            df = pd.read_csv(os.path.join(mass_dir, f))
            r = df["r_scaled"].values
            m = df["m_scaled"].values
            rvmax = compute_rvmax_from_mass_profile(r, m)

            halo_id = f.split("_")[1]
            if halo_id not in df_rvir["halo_id"].values:
                print(f"[Warning] halo_id {halo_id} not found in virial_radius.csv")
                continue
            rvir = df_rvir.loc[df_rvir.halo_id == halo_id, "rvir"].values[0]

            halo_ids.append(halo_id)
            rvmax_list.append(rvmax * rvir)

        except Exception as e:
            print(f"[Warning] Failed to process {f}: {e}")
            continue

    df_out = pd.DataFrame({"halo_id": halo_ids, "rvmax": rvmax_list})
    df_out.to_csv(os.path.join(output_dir, "max_radius.csv"), index=False)
    print(f"[Saved] max_radius.csv to {output_dir}/max_radius.csv")

save_rvmax_from_mass_profile(base_dir="/Users/fengbocheng/Projects/Symphony-PPSD", suite_name="SymphonyCluster")

In [ ]:
def compute_and_save_jeans_deviation(base_dir, suite_name):
    """
    Compute the Jeans equation deviation δ_J(r) for each halo and save to CSV.
    NOTE: G=1 !!!
    """
    density_dir = os.path.join(base_dir, "output", suite_name, "density_profiles")
    velocity_dir = os.path.join(base_dir, "output", suite_name, "velocity_profiles")
    mass_dir = os.path.join(base_dir, "output", suite_name, "mass_profiles")
    out_dir = os.path.join(base_dir, "output", suite_name, "jeans_deviation")
    os.makedirs(out_dir, exist_ok=True)

    halo_files = sorted([f for f in os.listdir(density_dir) if f.endswith(".csv")])

    for f in halo_files:
        halo_id = f.split("_")[1]
        try:
            df_rho = pd.read_csv(os.path.join(density_dir, f))
            df_vel = pd.read_csv(os.path.join(velocity_dir, f))
            df_mass = pd.read_csv(os.path.join(mass_dir, f))

            r = df_rho["r_scaled"].values
            rho = df_rho["rho_scaled"].values
            sigma_r2 = df_vel["sigma_rad_scaled"].values ** 2
            beta = df_vel["beta"].values
            m = df_mass["m_scaled"].values

            # Compute pressure gradient d/dr (rho * sigma_r^2)
            P_r = rho * sigma_r2
            dPdr = np.gradient(P_r, r)

            # Compute dPhi/dr = G * M(<r) / r^2
            dPhidr = m / (r ** 2)

            # Compute numerator and denominator
            numer = dPdr + 2 * beta * P_r / r + rho * dPhidr
            denom = rho * dPhidr

            delta_J = np.abs(numer / denom)

            df_out = pd.DataFrame({
                "halo_id": int(halo_id),
                "r_scaled": r,
                "delta_J": delta_J
            })
            df_out.to_csv(os.path.join(out_dir, f"halo_{int(halo_id):03d}_profile.csv"), index=False)
            print(f"[Saved] Jeans deviation for halo {halo_id}")
        except Exception as e:
            print(f"[Warning] Failed to compute Jeans deviation for halo {halo_id}: {e}")

base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
compute_and_save_jeans_deviation(base_dir, "SymphonyCluster")

In [ ]:
def compute_delta_J_tot(r, delta_J):
    mask = r <= 1.0
    r_cut = r[mask]
    delta_J_cut = delta_J[mask]
    delta_J_tot = np.trapezoid(delta_J_cut, r_cut)
    return delta_J_tot

def compute_and_save_delta_J_tot_per_suite(base_dir, suite_names, jeans_dir_name="jeans_deviation"):
    for suite in suite_names:
        results = []
        jeans_dir = os.path.join(base_dir, "output", suite, jeans_dir_name)
        files = sorted([f for f in os.listdir(jeans_dir) if f.endswith(".csv")])

        for f in files:
            halo_id = int(f.split("_")[1])
            df = pd.read_csv(os.path.join(jeans_dir, f))
            r = df["r_scaled"].values
            delta_J = df["delta_J"].values

            delta_J_tot = compute_delta_J_tot(r, delta_J)

            results.append({
                "halo_id": halo_id,
                "delta_J_tot": delta_J_tot
            })

        df_summary = pd.DataFrame(results)

        output_csv = os.path.join(base_dir, "output", suite, f"jeans_deviation_total.csv")
        df_summary.to_csv(output_csv, index=False)
        print(f"[Saved] Jeans deviation total summary for {suite} to {output_csv}")

base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
suite_names = ["SymphonyLMC", "SymphonyMilkyWay", "SymphonyGroup", "SymphonyLCluster", "SymphonyCluster"]

compute_and_save_delta_J_tot_per_suite(base_dir, suite_names)

In [ ]:
def save_accretion_rates(base_dir, suite_name, output):

    # Create output directory
    output_dir = os.path.join(output, "output", suite_name)
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"accretion_rates.csv")

    # Open file for writing
    with open(output_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["halo_id", "gamma"])
        n_halos = symlib.n_hosts(suite_name)
        i = 0
        while True:
            try:
                sim_dir = symlib.get_host_directory(base_dir, suite_name, i)
            except:
                break  # reached end of halo list

            try:
                scale = symlib.scale_factors(sim_dir)
                r, _ = symlib.read_rockstar(sim_dir)
                snap = len(scale) - 1
                m_now = r[0, snap]["m"]

                sim_params = symlib.simulation_parameters(sim_dir)
                cosmo = cosmology.setCosmology("custom", {
                    "flat": sim_params["flat"],
                    "H0": sim_params["H0"],
                    "Om0": sim_params["Om0"],
                    "Ob0": sim_params["Ob0"],
                    "sigma8": sim_params["sigma8"],
                    "ns": sim_params["ns"]
                })

                rho_m0 = cosmo.rho_m(0)
                Delta = 99
                rho_vir = Delta * rho_m0 * 1e9 / sim_params["h100"]**2
                G_val = G.to(u.Mpc**3 / (u.Msun * u.Gyr**2)).value
                t_dyn = 1.0 / np.sqrt((4 / 3) * np.pi * G_val * rho_vir)

                times = cosmo.age(1 / scale - 1)
                t0 = times[snap]
                t_past = t0 - t_dyn
                snap_past = np.argmin(np.abs(times - t_past))
                m_past = r[0, snap_past]["m"]

                gamma = (m_now - m_past) / t_dyn
                writer.writerow([f'{i:03d}', gamma])

            except Exception as e:
                traceback.print_exc()
                warnings.warn(f"[{suite_name} - halo {i}] Error calculating accretion rate: {e}")
                writer.writerow([i, "nan"])  # still record halo_index

            i += 1

    print(f"[Saved] {suite_name} accretion rates → {output_path}")

save_accretion_rates('/Volumes/Expansion/Symphony', 'SymphonyCluster', "/Users/fengbocheng/Projects/Symphony-PPSD")


In [ ]:
def save_concentration(base_dir, suite_name, output):
    output_dir = os.path.join(output, "output", suite_name)
    os.makedirs(output_dir, exist_ok=True)

    n_halos = symlib.n_hosts(suite_name)
    halo_ids, cvir_list = [], []

    for halo_idx in range(n_halos):
        sim_dir = symlib.get_host_directory(base_dir, suite_name, halo_idx)
        try:
            r, _ = symlib.read_rockstar(sim_dir)
            cvir = r[0, -1]["cvir"]
            halo_ids.append(f'{halo_idx:03d}')
            cvir_list.append(cvir)
        except FileNotFoundError:
            print(f"[Warning] Rockstar file not found for Halo {halo_idx}")
            continue

    df = pd.DataFrame({"halo_id": halo_ids, "cvir": cvir_list})
    df.to_csv(os.path.join(output_dir, "halo_concentrations.csv"), index=False)
    print(f"[Saved] Concentration CSV to {output_dir}/concentrations.csv")

In [ ]:
def save_mass(base_dir, suite_name, output):
    """
    Save halo mass data to a CSV file.
    """
    output_dir = os.path.join(output, "output", suite_name)
    os.makedirs(output_dir, exist_ok=True)

    n_halos = symlib.n_hosts(suite_name)
    halo_ids, mvir_list = [], []

    for halo_idx in range(n_halos):
        sim_dir = symlib.get_host_directory(base_dir, suite_name, halo_idx)
        try:
            r, _ = symlib.read_rockstar(sim_dir)
            mvir = r[0, -1]["m"] 
            halo_ids.append(f'{halo_idx:03d}')
            mvir_list.append(mvir)
        except FileNotFoundError:
            print(f"[Warning] Rockstar file not found for Halo {halo_idx}")
            continue

    df = pd.DataFrame({"halo_id": halo_ids, "mvir": mvir_list})
    df.to_csv(os.path.join(output_dir, "halo_mass.csv"), index=False)
    print(f"[Saved] Mass CSV to {output_dir}/halo_mass.csv")
